# OSHPD Hospital Data Cleaning Project

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import scipy
from scipy import stats
import plotly.express as px
import re

**Goal**: Our team compiled 2019 common hospital procedure average cost information submitted to California's OSHPD into a single dataset. This code attempts to systematically clean the cost information. 

**Main Challenges to Overcome:**

1) Hospitals submitted data in different formats. Some observations included: $ , ; - etc

2) Some hospitals included more than one value per cell

3) Some hospitals included cents and some did not. Since working with strings, we have to be cautious not lose this information


In [2]:
####Hospital Data Set
original_Med_Hacks_df = pd.read_csv("/Users/james/Desktop/Medhacks/Full_AB_1045_Final_3.csv")
original_Med_Hacks_df.head()

,Unnamed: 0,CPT_CODE_OR_OTHER,Top_25_Avg,OSHPD_NUM,Hospital_Name
0,Evaluation & Management Services (CPT Codes 99...,NaN,NaN,NaN,NaN
1,"Emergency Room Visit, Level 2 (low to moderate...",99282,$799.84,106010735.0,Alameda Hospital
2,"Emergency Room Visit, Level 3 (moderate severity)",99283,"$1,267.70",106010735.0,Alameda Hospital
3,"Emergency Room Visit, Level 4 (high severity)",99284,"$1,833.07",106010735.0,Alameda Hospital
4,"Outpatient Visit, established patient, 15 minutes",99213,$562.69,106010735.0,Alameda Hospital


In [3]:
###Change variable name to begin cleaning process
Med_Hacks_df=original_Med_Hacks_df

## Step 1: Remove NA's  and Make Hospital Numbers Integers

In [4]:
####Double Check for cells with more than one .
#Med_Hacks_df_let_per= Med_Hacks_df[Med_Hacks_df['Top_25_Avg'].str.contains('[.]+', na=False)] 
#Med_Hacks_df_let_per

In [5]:
#####Drop rows where Top_25_Avg is NaN
Med_Hacks_df = Med_Hacks_df.dropna(subset=['Top_25_Avg'])

In [6]:
###Change OSHPD Number to Integer
Med_Hacks_df['OSHPD_NUM'] = Med_Hacks_df['OSHPD_NUM'].astype(int)

/Users/james/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


## Step 2: Find and remove cells that include more than 1 value per cell
We will search the data for indications of more than 1 value like ; or / or - and remove problematic observations. 

In [7]:
####Visually inspected data to get the OSHPD number for hospitals that included cells with more than one value
#Use code below to view rows and strings to search for. 
#Med_Hacks_df[Med_Hacks_df['OSHPD_NUM']==106220773]
#Med_Hacks_df[Med_Hacks_df['OSHPD_NUM']==106304113]
#Med_Hacks_df[Med_Hacks_df['OSHPD_NUM']==106190034]

In [8]:
####Double Check for Other cells including semi colon
Med_Hacks_df_let_semi= Med_Hacks_df[Med_Hacks_df['Top_25_Avg'].str.contains('[;]', na=False)]   
Med_Hacks_df_let_semi

,Unnamed: 0,CPT_CODE_OR_OTHER,Top_25_Avg,OSHPD_NUM,Hospital_Name
7046,"Urinalysis, without microscopy",81002 or 81003,$77; $0,106300032,CHOC CHILDREN'S HOSPITAL OF ORANGE COUNTY
7047,"Urinalysis, with microscopy",81000 or 81001,$0; $218,106300032,CHOC CHILDREN'S HOSPITAL OF ORANGE COUNTY
7063,"Physical Therapy, Evaluation",97161-97163,$867; $927;$802,106300032,CHOC CHILDREN'S HOSPITAL OF ORANGE COUNTY
8279,"Urinalysis, without microscopy",81002 or 81003,$0; $142,106304113,CHOC Children's at Mission
8280,"Urinalysis, with microscopy",81000 or 81001,$0; $166,106304113,CHOC Children's at Mission
8296,"Physical Therapy, Evaluation",97161-97163,$290; $395; $395,106304113,CHOC Children's at Mission


In [9]:
####Double Check for Other cells including backslash

Med_Hacks_df_let_backslash= Med_Hacks_df[Med_Hacks_df['Top_25_Avg'].str.contains('[/]', na=False)] 
Med_Hacks_df_let_backslash

,Unnamed: 0,CPT_CODE_OR_OTHER,Top_25_Avg,OSHPD_NUM,Hospital_Name
12443,Other Common Outpatient Procedures (list as ne...,2019 CPT CODE,Average Charge /Case,106434040,Lucile Salter Packard Children's Hospital
13078,"Physical Therapy, Evaluation",97161-97163,313.00/399.00/486.00,106490964,Healdsburg District Hospital
16296,"Urinalysis, without microscopy",81002 or 81003,22.00/19.00,106190034,Antelope Valley Hospital
16297,"Urinalysis, with microscopy",81000 or 81001,NA/27.00,106190034,Antelope Valley Hospital
16313,"Physical Therapy, Evaluation",97161-97163,344.00/394.00/444.00,106190034,Antelope Valley Hospital
16329,"Injection, Diagnostic or Therapeutic substance...",62322-62323,3084.63 / 4871.63,106190034,Antelope Valley Hospital


In [10]:
####Double Check for Other cells including dash
Med_Hacks_df['Top_25_Avg']=Med_Hacks_df['Top_25_Avg'].str.replace('\t-', '')
Med_Hacks_df_let_dash= Med_Hacks_df[Med_Hacks_df['Top_25_Avg'].str.contains('[-]+', na=False)]   
Med_Hacks_df_let_dash

/Users/james/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,Unnamed: 0,CPT_CODE_OR_OTHER,Top_25_Avg,OSHPD_NUM,Hospital_Name
8558,"Physical Therapy, Evaluation",97161-97163,$106-$212,106320859,EASTERN PLUMAS HEALTH CARE DISTRICT
16149,"Physical Therapy, Evaluation",97161-97163,$307.00-$470.00,106190176,CITY OF HOPE


In [11]:
###Dealing with rows with multiple values in Top_25_Avg cell. We observe it appears each value corresponds 
#to a CPT code in a range.

problem_rows=Med_Hacks_df.loc[[6311, 7046, 7047, 7063,8279, 8280,8296,8558,13078,16296,16297,16313,16329,16149], :]
problem_rows

,Unnamed: 0,CPT_CODE_OR_OTHER,Top_25_Avg,OSHPD_NUM,Hospital_Name
6311,"Physical Therapy, Evaluation",97161-97163,"$86, $129, $161",106220773,John C. Fremont Healthcare District
7046,"Urinalysis, without microscopy",81002 or 81003,$77; $0,106300032,CHOC CHILDREN'S HOSPITAL OF ORANGE COUNTY
7047,"Urinalysis, with microscopy",81000 or 81001,$0; $218,106300032,CHOC CHILDREN'S HOSPITAL OF ORANGE COUNTY
7063,"Physical Therapy, Evaluation",97161-97163,$867; $927;$802,106300032,CHOC CHILDREN'S HOSPITAL OF ORANGE COUNTY
8279,"Urinalysis, without microscopy",81002 or 81003,$0; $142,106304113,CHOC Children's at Mission
8280,"Urinalysis, with microscopy",81000 or 81001,$0; $166,106304113,CHOC Children's at Mission
8296,"Physical Therapy, Evaluation",97161-97163,$290; $395; $395,106304113,CHOC Children's at Mission
8558,"Physical Therapy, Evaluation",97161-97163,$106-$212,106320859,EASTERN PLUMAS HEALTH CARE DISTRICT
13078,"Physical Therapy, Evaluation",97161-97163,313.00/399.00/486.00,106490964,Healdsburg District Hospital
16296,"Urinalysis, without microscopy",81002 or 81003,22.00/19.00,106190034,Antelope Valley Hospital


In [12]:
####Drop problematic rows
Med_Hacks_df = Med_Hacks_df.drop([6311, 7046, 7047, 7063,8279, 8280,8296,8558,13078,16296,16297,16313,16329,16149])

## Step 3: Clean Average Cost Data
We start by seperating cents into a seperate column so that we can simply remove all non-numeric values without losing the cents information. 

In [13]:
#Since some costs include cents and some do not. We will make a seperate column for cents by using the . 
#as a delimineator 

# new data frame with split value columns. This code splits at the first . 
new = Med_Hacks_df["Top_25_Avg"].str.split(".", n = 1, expand = True) 
#making separate column with just dollars from new data frame 
Med_Hacks_df["Top_25_Avg"]=new[0]
#making separate column with just cents from new data frame 
Med_Hacks_df["Top_25_Avg_cents"]=new[1]



In [14]:
####Since we are splitting at the first period (n=1), Double check to make sure no .'s remain in Top_25_Avg:
Med_Hacks_df_let_period = Med_Hacks_df[Med_Hacks_df['Top_25_Avg'].str.contains('[.]', na=False)]    
Med_Hacks_df_let_period

,Unnamed: 0,CPT_CODE_OR_OTHER,Top_25_Avg,OSHPD_NUM,Hospital_Name,Top_25_Avg_cents


In [15]:
Med_Hacks_df

,Unnamed: 0,CPT_CODE_OR_OTHER,Top_25_Avg,OSHPD_NUM,Hospital_Name,Top_25_Avg_cents
1,"Emergency Room Visit, Level 2 (low to moderate...",99282,$799,106010735,Alameda Hospital,84
2,"Emergency Room Visit, Level 3 (moderate severity)",99283,"$1,267",106010735,Alameda Hospital,70
3,"Emergency Room Visit, Level 4 (high severity)",99284,"$1,833",106010735,Alameda Hospital,07
4,"Outpatient Visit, established patient, 15 minutes",99213,$562,106010735,Alameda Hospital,69
6,Basic Metabolic Panel,80048,$218,106010735,Alameda Hospital,87
...,...,...,...,...,...,...
16816,"Injection, Diagnostic or Therapeutic substance...",62322-62323,"$5,295",106171049,Adventist Health Clearlake,04
16817,"Injection, Anesthetic or Steroid, transforamin...",64483,"$5,265",106171049,Adventist Health Clearlake,87
16818,Laparoscopic Cholecystectomy,47562,"$18,127",106171049,Adventist Health Clearlake,22
16819,"Tympanostomy (insert ventilating tube, genera...",69436,"$8,398",106171049,Adventist Health Clearlake,91


In [16]:
###Remove $ signs. This step is actually unnecessary since we are only going to keep numeric values.
Med_Hacks_df['Top_25_Avg']=Med_Hacks_df['Top_25_Avg'].str.replace('$', '')

In [17]:
#####See how many rows contain a comma
Med_Hacks_df_let_comma= Med_Hacks_df[Med_Hacks_df['Top_25_Avg'].str.contains('[,]', na=False)]   

In [18]:
Med_Hacks_df_let_comma

,Unnamed: 0,CPT_CODE_OR_OTHER,Top_25_Avg,OSHPD_NUM,Hospital_Name,Top_25_Avg_cents
2,"Emergency Room Visit, Level 3 (moderate severity)",99283,"1,267",106010735,Alameda Hospital,70
3,"Emergency Room Visit, Level 4 (high severity)",99284,"1,833",106010735,Alameda Hospital,07
20,"CT Scan, Abdomen, with contrast",74160,"4,107",106010735,Alameda Hospital,00
21,"CT Scan, Head or Brain, without contrast",70450,"3,482",106010735,Alameda Hospital,82
22,"CT Scan, Pelvis, with contrast",72193,"4,210",106010735,Alameda Hospital,00
...,...,...,...,...,...,...
16815,"Hernia Repair, Inguinal, 5 years and older",49505,"27,052",106171049,Adventist Health Clearlake,26
16816,"Injection, Diagnostic or Therapeutic substance...",62322-62323,"5,295",106171049,Adventist Health Clearlake,04
16817,"Injection, Anesthetic or Steroid, transforamin...",64483,"5,265",106171049,Adventist Health Clearlake,87
16818,Laparoscopic Cholecystectomy,47562,"18,127",106171049,Adventist Health Clearlake,22


In [19]:
###Remove all commas. This step is actually unnecessary since we are only going to keep numeric values.
Med_Hacks_df['Top_25_Avg']=Med_Hacks_df['Top_25_Avg'].str.replace(',', '')

In [20]:
###Remove all "Average Charge". This step is actually unnecessary since we are only going to keep numeric values.
Med_Hacks_df['Top_25_Avg']=Med_Hacks_df['Top_25_Avg'].str.replace('Average Charge', '')

In [21]:
Med_Hacks_df.head()

,Unnamed: 0,CPT_CODE_OR_OTHER,Top_25_Avg,OSHPD_NUM,Hospital_Name,Top_25_Avg_cents
1,"Emergency Room Visit, Level 2 (low to moderate...",99282,799,106010735,Alameda Hospital,84
2,"Emergency Room Visit, Level 3 (moderate severity)",99283,1267,106010735,Alameda Hospital,70
3,"Emergency Room Visit, Level 4 (high severity)",99284,1833,106010735,Alameda Hospital,07
4,"Outpatient Visit, established patient, 15 minutes",99213,562,106010735,Alameda Hospital,69
6,Basic Metabolic Panel,80048,218,106010735,Alameda Hospital,87


In [22]:
Med_Hacks_df.shape

(10784, 6)

In [23]:
####Double check to make sure we will not get rid of anything too significant by keeping numbers only:
Med_Hacks_df_let_string = Med_Hacks_df[Med_Hacks_df['Top_25_Avg'].str.contains('[A-Za-z]', na=False)]    
Med_Hacks_df_let_string['Top_25_Avg'].value_counts()

Outpatient Charge Amount    25
 Amount                     21
SCAL                         5
Ave Charge                   5
 /Case                       1
  Amount                     1
Not Reported                 1
AVERAGE CHARGE               1
Name: Top_25_Avg, dtype: int64

In [24]:
###Method 1: Remove all parts of the string that are not numbers. 
df = Med_Hacks_df.copy()
def keepdigit(string):
    """
    input is a string 
    output is a string with no digits
    """
    return ''.join(ch for ch in string if ch.isdigit())
df.Top_25_Avg = df.Top_25_Avg.apply(keepdigit)
df

,Unnamed: 0,CPT_CODE_OR_OTHER,Top_25_Avg,OSHPD_NUM,Hospital_Name,Top_25_Avg_cents
1,"Emergency Room Visit, Level 2 (low to moderate...",99282,799,106010735,Alameda Hospital,84
2,"Emergency Room Visit, Level 3 (moderate severity)",99283,1267,106010735,Alameda Hospital,70
3,"Emergency Room Visit, Level 4 (high severity)",99284,1833,106010735,Alameda Hospital,07
4,"Outpatient Visit, established patient, 15 minutes",99213,562,106010735,Alameda Hospital,69
6,Basic Metabolic Panel,80048,218,106010735,Alameda Hospital,87
...,...,...,...,...,...,...
16816,"Injection, Diagnostic or Therapeutic substance...",62322-62323,5295,106171049,Adventist Health Clearlake,04
16817,"Injection, Anesthetic or Steroid, transforamin...",64483,5265,106171049,Adventist Health Clearlake,87
16818,Laparoscopic Cholecystectomy,47562,18127,106171049,Adventist Health Clearlake,22
16819,"Tympanostomy (insert ventilating tube, genera...",69436,8398,106171049,Adventist Health Clearlake,91


In [25]:
####Method 2: Same process with Regular Expressions
###Use RE to remove all none digits with /D
#df = Med_Hacks_df.copy()
#df.Top_25_Avg = df.Top_25_Avg.str.replace('\D','').str.strip()
#df.Top_25_Avg.head()

In [26]:
####Remove any empty cells in Top_25_Avg column
df=df[df.Top_25_Avg != '']

In [27]:
#df['Top_25_Avg'] = pd.to_numeric(df['Top_25_Avg'],errors='coerce')
df['Top_25_Avg'] = df['Top_25_Avg'].astype(float)

/Users/james/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [28]:
df.dtypes

Unnamed: 0            object
CPT_CODE_OR_OTHER     object
Top_25_Avg           float64
OSHPD_NUM              int64
Hospital_Name         object
Top_25_Avg_cents      object
dtype: object

In [29]:
df

,Unnamed: 0,CPT_CODE_OR_OTHER,Top_25_Avg,OSHPD_NUM,Hospital_Name,Top_25_Avg_cents
1,"Emergency Room Visit, Level 2 (low to moderate...",99282,799.0,106010735,Alameda Hospital,84
2,"Emergency Room Visit, Level 3 (moderate severity)",99283,1267.0,106010735,Alameda Hospital,70
3,"Emergency Room Visit, Level 4 (high severity)",99284,1833.0,106010735,Alameda Hospital,07
4,"Outpatient Visit, established patient, 15 minutes",99213,562.0,106010735,Alameda Hospital,69
6,Basic Metabolic Panel,80048,218.0,106010735,Alameda Hospital,87
...,...,...,...,...,...,...
16815,"Hernia Repair, Inguinal, 5 years and older",49505,27052.0,106171049,Adventist Health Clearlake,26
16816,"Injection, Diagnostic or Therapeutic substance...",62322-62323,5295.0,106171049,Adventist Health Clearlake,04
16817,"Injection, Anesthetic or Steroid, transforamin...",64483,5265.0,106171049,Adventist Health Clearlake,87
16818,Laparoscopic Cholecystectomy,47562,18127.0,106171049,Adventist Health Clearlake,22


## Step 4: Clean cents column and add back to cleaned Top_25_Avg

In [30]:
cents=pd.DataFrame(df['Top_25_Avg_cents'].value_counts())
cents.reset_index(drop=False, inplace=True)
cents.columns =['Top_25_Avg_cents','Freq'] 
cents

,Top_25_Avg_cents,Freq
0,00,5106
1,50,165
2,33,92
3,90,78
4,75,70
...,...,...
104,8,2
105,.30,1
106,3,1
107,6,1


In [31]:
###Check to see if any characters made their way into the cents column by removing numbers

df_cents = cents.copy()
def removedigit(string):
    """
    input is a string 
    output is a string with no digits
    """
    return ''.join(ch for ch in string if not ch.isdigit())
df_cents.Top_25_Avg_cents = df_cents.Top_25_Avg_cents.apply(removedigit)
df_cents[df_cents['Top_25_Avg_cents']!='']

,Top_25_Avg_cents,Freq
105,.,1


In [32]:
####Remove . from Top_25_Avg_cents column
df['Top_25_Avg_cents']=df['Top_25_Avg_cents'].str.replace('.', '')
###Replace None with 0
df['Top_25_Avg_cents'].replace(pd.np.nan, 0, inplace=True)



/Users/james/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/Users/james/opt/anaconda3/lib/python3.7/site-packages/pandas/core/generic.py:6786: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._update_inplace(new_data)


In [33]:
#df['Top_25_Avg_cents']=df['Top_25_Avg_cents'].str.replace('None', '0')
df[df['OSHPD_NUM']==106010937]

,Unnamed: 0,CPT_CODE_OR_OTHER,Top_25_Avg,OSHPD_NUM,Hospital_Name,Top_25_Avg_cents
283,"Emergency Room Visit, Level 2",99282,1077.0,106010937,Summit Medical Center Merritt Campus,0
284,"Emergency Room Visit, Level 3",99283,2060.0,106010937,Summit Medical Center Merritt Campus,0
285,"Emergency Room Visit, Level 4",99284,3490.0,106010937,Summit Medical Center Merritt Campus,0
287,Basic Metabolic Panel,80048,170.0,106010937,Summit Medical Center Merritt Campus,0
288,Comprehensive Metabolic Panel,80053,198.0,106010937,Summit Medical Center Merritt Campus,0
289,Lipid Panel,80061,224.0,106010937,Summit Medical Center Merritt Campus,0
290,"Creatine Kinase (CK), (CPK), Total",82550,130.0,106010937,Summit Medical Center Merritt Campus,0
291,"Blood Gas Analysis, including 02 saturation",82805,818.0,106010937,Summit Medical Center Merritt Campus,0
292,Thyroid Stimulating Hormone,84443,185.0,106010937,Summit Medical Center Merritt Campus,0
293,"Troponin, Quantitative",84484,381.0,106010937,Summit Medical Center Merritt Campus,0


In [34]:
df['Top_25_Avg_cents'].value_counts()

00    5106
0     1010
50     165
33      92
90      78
      ... 
2        2
8        2
6        1
3        1
5        1
Name: Top_25_Avg_cents, Length: 109, dtype: int64

In [35]:
#Check to make sure no cents string greater than 2 digits
mask = (df['Top_25_Avg_cents'].str.len() > 2) 
df_cents_over2 = df.loc[mask]
print(df_cents_over2)

Empty DataFrame
Columns: [Unnamed: 0, CPT_CODE_OR_OTHER, Top_25_Avg, OSHPD_NUM, Hospital_Name, Top_25_Avg_cents]
Index: []


In [36]:
df['Top_25_Avg_cents']=df['Top_25_Avg_cents'].astype(float)/100
df[df['Top_25_Avg_cents']>1]

/Users/james/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,Unnamed: 0,CPT_CODE_OR_OTHER,Top_25_Avg,OSHPD_NUM,Hospital_Name,Top_25_Avg_cents


In [37]:
df['Top_25_Avg']=df['Top_25_Avg']+df['Top_25_Avg_cents']

/Users/james/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [38]:
del df['Top_25_Avg_cents']

In [39]:
df

,Unnamed: 0,CPT_CODE_OR_OTHER,Top_25_Avg,OSHPD_NUM,Hospital_Name
1,"Emergency Room Visit, Level 2 (low to moderate...",99282,799.84,106010735,Alameda Hospital
2,"Emergency Room Visit, Level 3 (moderate severity)",99283,1267.70,106010735,Alameda Hospital
3,"Emergency Room Visit, Level 4 (high severity)",99284,1833.07,106010735,Alameda Hospital
4,"Outpatient Visit, established patient, 15 minutes",99213,562.69,106010735,Alameda Hospital
6,Basic Metabolic Panel,80048,218.87,106010735,Alameda Hospital
...,...,...,...,...,...
16815,"Hernia Repair, Inguinal, 5 years and older",49505,27052.26,106171049,Adventist Health Clearlake
16816,"Injection, Diagnostic or Therapeutic substance...",62322-62323,5295.04,106171049,Adventist Health Clearlake
16817,"Injection, Anesthetic or Steroid, transforamin...",64483,5265.87,106171049,Adventist Health Clearlake
16818,Laparoscopic Cholecystectomy,47562,18127.22,106171049,Adventist Health Clearlake


In [40]:
#df.to_csv('AB_1045_Python_Cleaned.csv')

## Step 5: Run additional diagnostics to identify potential mistakes

In [41]:
df.describe()

,Top_25_Avg,OSHPD_NUM
count,9173.000000,9.173000e+03
mean,3445.349902,1.043449e+08
std,8688.050001,5.736640e+07
min,0.000000,9.079300e+04
25%,197.000000,1.061902e+08
50%,619.000000,1.063000e+08
75%,2684.820000,1.063708e+08
max,226413.000000,1.063051e+09


In [42]:
outliers_df=df[df['Top_25_Avg']>80000]
outliers_df

,Unnamed: 0,CPT_CODE_OR_OTHER,Top_25_Avg,OSHPD_NUM,Hospital_Name
2878,Insert new or replace of permanent pacemaker w...,33208,122912.00,106190315,AHMC GARFIELD MEDICAL CENTER
2879,Insert or replaceof permpacing cardi-defibril ...,33249,226413.00,106190315,AHMC GARFIELD MEDICAL CENTER
4193,INSERT MULTI-COMP PENIS PROS,54405,102289.00,106190547,AHMC MONTEREY PARK HOSPITAL
6910,"Hernia Repair, Inguinal, 5 years and older",49505,82109.38,106281078,Adventist Health St Helena
14155,"LAPAROSCOPY SURGICAL, WITH VAGINAL HYSTERECTOM...",58552,105832.68,106564121,Los Robles Hospital - Thousand Oaks Surgical H...
14157,"LAMINOTOMY (HEMILAMINECTOMY), W DECOMPRESSION ...",63030,84974.00,106564121,Los Robles Hospital - Thousand Oaks Surgical H...
14159,REDUCTION MAMMAPLASTY,19318,89790.47,106564121,Los Robles Hospital - Thousand Oaks Surgical H...
14160,"ARTHROPLASTY, KNEE, CONDYLE AND PLATEAU; MEDIA...",27446,136084.94,106564121,Los Robles Hospital - Thousand Oaks Surgical H...
14162,DELAYED INSERTION OF BREAST PROSTHESIS FOLLOWI...,19342,121478.81,106564121,Los Robles Hospital - Thousand Oaks Surgical H...
14165,IMMEDIATE INSERTION OF BREAST PROSTHESIS FOLLO...,19340,88272.00,106564121,Los Robles Hospital - Thousand Oaks Surgical H...


In [43]:
### Use orginal data to review potential outliers


outlier_rows=original_Med_Hacks_df.loc[[2878, 2879,4193,6910,14155,14157,14159,14160,14162,14165,14167,14168,14169,14177], :]
outlier_rows

,Unnamed: 0,CPT_CODE_OR_OTHER,Top_25_Avg,OSHPD_NUM,Hospital_Name
2878,Insert new or replace of permanent pacemaker w...,33208,"$122,912.00",106190315.0,AHMC GARFIELD MEDICAL CENTER
2879,Insert or replaceof permpacing cardi-defibril ...,33249,"$226,413.00",106190315.0,AHMC GARFIELD MEDICAL CENTER
4193,INSERT MULTI-COMP PENIS PROS,54405,"$102,289.00",106190547.0,AHMC MONTEREY PARK HOSPITAL
6910,"Hernia Repair, Inguinal, 5 years and older",49505,"$82,109.38",106281078.0,Adventist Health St Helena
14155,"LAPAROSCOPY SURGICAL, WITH VAGINAL HYSTERECTOM...",58552,"$105,832.68",106564121.0,Los Robles Hospital - Thousand Oaks Surgical H...
14157,"LAMINOTOMY (HEMILAMINECTOMY), W DECOMPRESSION ...",63030,"$84,974.00",106564121.0,Los Robles Hospital - Thousand Oaks Surgical H...
14159,REDUCTION MAMMAPLASTY,19318,"$89,790.47",106564121.0,Los Robles Hospital - Thousand Oaks Surgical H...
14160,"ARTHROPLASTY, KNEE, CONDYLE AND PLATEAU; MEDIA...",27446,"$136,084.94",106564121.0,Los Robles Hospital - Thousand Oaks Surgical H...
14162,DELAYED INSERTION OF BREAST PROSTHESIS FOLLOWI...,19342,"$121,478.81",106564121.0,Los Robles Hospital - Thousand Oaks Surgical H...
14165,IMMEDIATE INSERTION OF BREAST PROSTHESIS FOLLO...,19340,"$88,272.00",106564121.0,Los Robles Hospital - Thousand Oaks Surgical H...


In [44]:
df[df['OSHPD_NUM']==106010937]

,Unnamed: 0,CPT_CODE_OR_OTHER,Top_25_Avg,OSHPD_NUM,Hospital_Name
283,"Emergency Room Visit, Level 2",99282,1077.0,106010937,Summit Medical Center Merritt Campus
284,"Emergency Room Visit, Level 3",99283,2060.0,106010937,Summit Medical Center Merritt Campus
285,"Emergency Room Visit, Level 4",99284,3490.0,106010937,Summit Medical Center Merritt Campus
287,Basic Metabolic Panel,80048,170.0,106010937,Summit Medical Center Merritt Campus
288,Comprehensive Metabolic Panel,80053,198.0,106010937,Summit Medical Center Merritt Campus
289,Lipid Panel,80061,224.0,106010937,Summit Medical Center Merritt Campus
290,"Creatine Kinase (CK), (CPK), Total",82550,130.0,106010937,Summit Medical Center Merritt Campus
291,"Blood Gas Analysis, including 02 saturation",82805,818.0,106010937,Summit Medical Center Merritt Campus
292,Thyroid Stimulating Hormone,84443,185.0,106010937,Summit Medical Center Merritt Campus
293,"Troponin, Quantitative",84484,381.0,106010937,Summit Medical Center Merritt Campus


# Step 6: Add Consumer Friendly Labels to Procedure Descriptions
**Strategy**: We will find only unique procedure descriptions/CPT codes using groupby and label them before merging them with the original data. 

Note: We used R to merge our hospital data with addresses. We will keep track of rows that are NaN for CPT code or Procedure descrition so as not to lose them when merging. 

In [45]:
####Hospital Data Set
Med_Hacks_df_updated = pd.read_csv("/Users/james/Desktop/Medhacks/AB_1045_merged_with_address_Final_2.csv")
Med_Hacks_df_updated.head()

,Unnamed: 0,OSHPD_NUM,X1.x,Procedure Description,CPT_CODE_OR_OTHER,Top_25_Avg,Hospital_Name.x,X1.y,Hospital_Name.y,Freq,Full_Address,Address,City,State,Zip Code
0,1,90793,1393,"Emergency Room Visit, Level 2 (low to moderate...",99282,1144.79,Barton Memorial Hospital,42,Barton Memorial Hospital,56.0,"2170 South Ave, South Lake Tahoe, CA",2170 South Ave,South Lake Tahoe,CA,96150
1,2,90793,1394,"Emergency Room Visit, Level 3 (moderate severity)",99283,1604.97,Barton Memorial Hospital,42,Barton Memorial Hospital,56.0,"2170 South Ave, South Lake Tahoe, CA",2170 South Ave,South Lake Tahoe,CA,96150
2,3,90793,1395,"Emergency Room Visit, Level 4 (high severity w...",99284,3308.75,Barton Memorial Hospital,42,Barton Memorial Hospital,56.0,"2170 South Ave, South Lake Tahoe, CA",2170 South Ave,South Lake Tahoe,CA,96150
3,4,90793,1396,"Emergency Room Visit, Level 4 (high severity w...",99285,4771.15,Barton Memorial Hospital,42,Barton Memorial Hospital,56.0,"2170 South Ave, South Lake Tahoe, CA",2170 South Ave,South Lake Tahoe,CA,96150
4,5,90793,1397,"Outpatient Visit, established patient, 15 minutes",99213,395.24,Barton Memorial Hospital,42,Barton Memorial Hospital,56.0,"2170 South Ave, South Lake Tahoe, CA",2170 South Ave,South Lake Tahoe,CA,96150


**Note**: To make categorizing easier, we create a data frame grouped by CPT Code and Procedure Description. This will allow us to more easily label distinct procedures and then merge with master data.

In [46]:
###Grouping by CPT and Procedure Description
procedures = Med_Hacks_df_updated.groupby(['CPT_CODE_OR_OTHER','Procedure Description'])['Top_25_Avg'].mean()
procedures=pd.DataFrame(procedures)
procedures.reset_index(drop=False, inplace=True)
print(procedures)

    CPT_CODE_OR_OTHER                              Procedure Description  \
0              0G0121  COLORECTAL CANCER SCREENING; COLONOSCOPY ON IN...   
1               10005  Fine Needle Aspiration with Imaging Guidance (...   
2               10022                                        FNA W/IMAGE   
3               10060                           Drainage of Skin Abscess   
4               10060  INCISION AND DRAINAGE OF ABSCESS (EG, CARBUNCL...   
..                ...                                                ...   
856             G0410                                  GRP PSYCH PARTIAL   
857             G0410                       GRP PSYCH PARTIAL HOSP 45-50   
858             G0410                            PHP GROUP THERAPY 60MIN   
859             G0463                                   Outpatient Visit   
860       G0463/99213  Outpatient Visit, established patient, 15 minutes   

     Top_25_Avg  
0       6479.88  
1       1333.00  
2       8244.00  
3        682.95

In [47]:
###Since another dataset was previously labelled, we start by merging existing labels with our unlabelled procedure df. 
Procedures_cat = pd.read_csv("/Users/james/Desktop/Medhacks/Procedures_Updated.csv")
Procedures_cat=Procedures_cat.rename(columns = {'Procedure Group': 'Procedure Description'})
Procedures_cat.head()


,Unnamed: 0,Procedure Description,Freq,Procedure_Group_Cat,Pro_Cat,Body_Part
0,1,"*Emergency Room Visit, Level 2 (low to moderat...",1,New Procedure,"Emergency Room Visit, Level 2 (low to moderate...",NaN
1,2,"*Emergency Room Visit, Level 3 (moderate sever...",1,New Procedure,"Emergency Room Visit, Level 3 (moderate severity)",NaN
2,3,"*Emergency Room Visit, Level 4 (high severity)",1,New Procedure,"Emergency Room Visit, Level 4 (high severity w...",NaN
3,4,2D Echo TTE Complete,1,New Procedure,Echocardiography,NaN
4,5,ABD PARACENTESIS W/IMAGING,2,New Procedure,Abdominal Paracentesis with Imaging,NaN


In [48]:
result = pd.merge(procedures, Procedures_cat, how='left', on='Procedure Description')
result

,CPT_CODE_OR_OTHER,Procedure Description,Top_25_Avg,Unnamed: 0,Freq,Procedure_Group_Cat,Pro_Cat,Body_Part
0,0G0121,COLORECTAL CANCER SCREENING; COLONOSCOPY ON IN...,6479.88,160.0,1.0,New Procedure,NaN,NaN
1,10005,Fine Needle Aspiration with Imaging Guidance (...,1333.00,359.0,1.0,New Procedure,NaN,NaN
2,10022,FNA W/IMAGE,8244.00,361.0,1.0,New Procedure,NaN,NaN
3,10060,Drainage of Skin Abscess,682.95,237.0,1.0,New Procedure,Drainage of Skin Abscess,NaN
4,10060,"INCISION AND DRAINAGE OF ABSCESS (EG, CARBUNCL...",7571.27,445.0,1.0,New Procedure,NaN,NaN
...,...,...,...,...,...,...,...,...
856,G0410,GRP PSYCH PARTIAL,350.00,404.0,2.0,New Procedure,GROUP THERAPY,NaN
857,G0410,GRP PSYCH PARTIAL HOSP 45-50,200.00,405.0,1.0,New Procedure,GROUP THERAPY,NaN
858,G0410,PHP GROUP THERAPY 60MIN,482.00,637.0,1.0,New Procedure,GROUP THERAPY,NaN
859,G0463,Outpatient Visit,362.00,610.0,5.0,New Procedure,NaN,NaN


In [49]:
##Exporting Data to Label
#result.to_csv('procedures_final2.csv')

## Import Labeled Data Frame and Merge with Master Dataset

In [50]:
Procedures_cat_updated = pd.read_csv("/Users/james/Desktop/Medhacks/procedures_final3.csv")

In [51]:
Procedures_cat_updated

,Unnamed: 0,CPT_CODE_OR_OTHER,Procedure Description,Freq,Procedure_Group_Cat,Pro_Cat
0,1,10005,Fine Needle Aspiration with Imaging Guidance (...,1.0,New Procedure,NaN
1,2,10022,FNA W/IMAGE,1.0,New Procedure,NaN
2,3,10060,Drainage of Skin Abscess,1.0,New Procedure,Drainage of Skin Abscess
3,4,10060,"INCISION AND DRAINAGE OF ABSCESS (EG, CARBUNCL...",1.0,New Procedure,NaN
4,5,11042,"DEBRIDEMENT; SKIN, AND SUBCUTANEOUS TISSUE",1.0,New Procedure,NaN
...,...,...,...,...,...,...
856,856,G0410,GRP PSYCH PARTIAL,2.0,New Procedure,GROUP THERAPY
857,857,G0410,GRP PSYCH PARTIAL HOSP 45-50,1.0,New Procedure,GROUP THERAPY
858,858,G0410,PHP GROUP THERAPY 60MIN,1.0,New Procedure,GROUP THERAPY
859,859,G0463,Outpatient Visit,5.0,New Procedure,NaN


In [52]:
##Make Missing Values Miscellaneous
Procedures_cat_updated['Pro_Cat'] = Procedures_cat_updated['Pro_Cat'].fillna("Miscellaneous")

In [53]:
Procedures_cat_updated

,Unnamed: 0,CPT_CODE_OR_OTHER,Procedure Description,Freq,Procedure_Group_Cat,Pro_Cat
0,1,10005,Fine Needle Aspiration with Imaging Guidance (...,1.0,New Procedure,Miscellaneous
1,2,10022,FNA W/IMAGE,1.0,New Procedure,Miscellaneous
2,3,10060,Drainage of Skin Abscess,1.0,New Procedure,Drainage of Skin Abscess
3,4,10060,"INCISION AND DRAINAGE OF ABSCESS (EG, CARBUNCL...",1.0,New Procedure,Miscellaneous
4,5,11042,"DEBRIDEMENT; SKIN, AND SUBCUTANEOUS TISSUE",1.0,New Procedure,Miscellaneous
...,...,...,...,...,...,...
856,856,G0410,GRP PSYCH PARTIAL,2.0,New Procedure,GROUP THERAPY
857,857,G0410,GRP PSYCH PARTIAL HOSP 45-50,1.0,New Procedure,GROUP THERAPY
858,858,G0410,PHP GROUP THERAPY 60MIN,1.0,New Procedure,GROUP THERAPY
859,859,G0463,Outpatient Visit,5.0,New Procedure,Miscellaneous


In [54]:
###Merge Labled data with master dataset
pro_cat_merged = pd.merge(Med_Hacks_df_updated, Procedures_cat_updated, how='inner', on=['Procedure Description','CPT_CODE_OR_OTHER'])
pro_cat_merged

,Unnamed: 0_x,OSHPD_NUM,X1.x,Procedure Description,CPT_CODE_OR_OTHER,Top_25_Avg,Hospital_Name.x,X1.y,Hospital_Name.y,Freq_x,Full_Address,Address,City,State,Zip Code,Unnamed: 0_y,Freq_y,Procedure_Group_Cat,Pro_Cat
0,1,90793,1393,"Emergency Room Visit, Level 2 (low to moderate...",99282,1144.79,Barton Memorial Hospital,42,Barton Memorial Hospital,56.0,"2170 South Ave, South Lake Tahoe, CA",2170 South Ave,South Lake Tahoe,CA,96150,820,260.0,New Procedure,"Emergency Room Visit, Level 2 (low to moderate..."
1,31,90933,1449,"Emergency Room Visit, Level 2 (low to moderate...",99282,1169.36,MARSHALL MEDICAL CENTER,150,MARSHALL MEDICAL CENTER,56.0,"1100 Marshall Way, Placerville, CA",1100 Marshall Way,Placerville,CA,95667,820,260.0,New Procedure,"Emergency Room Visit, Level 2 (low to moderate..."
2,71,190434,3463,"Emergency Room Visit, Level 2 (low to moderate...",99282,942.00,Kaiser Foundation Hospital- West LA,122,Kaiser Foundation Hospital- West LA,56.0,"6041 Cadillac Ave, Los Angeles, CA",6041 Cadillac Ave,Los Angeles,CA,90034,820,260.0,New Procedure,"Emergency Room Visit, Level 2 (low to moderate..."
3,111,191230,5625,"Emergency Room Visit, Level 2 (low to moderate...",99282,3817.00,"MARTIN LUTHER KING JR, COMMUNITY HOSPITAL",151,"MARTIN LUTHER KING JR, COMMUNITY HOSPITAL",55.0,"1680 E 120th St, Los Angeles, CA",1680 E 120th St,Los Angeles,CA,90059,820,260.0,New Procedure,"Emergency Room Visit, Level 2 (low to moderate..."
4,124,191450,5706,"Emergency Room Visit, Level 2 (low to moderate...",99282,942.00,Kaiser Foundation Hospital- Woodland Hills,123,Kaiser Foundation Hospital- Woodland Hills,56.0,"5601 De Soto Ave, Woodland Hills, CA",5601 De Soto Ave,Woodland Hills,CA,91367,820,260.0,New Procedure,"Emergency Room Visit, Level 2 (low to moderate..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9132,9039,106564121,14171,"ARTHROSCOPY, KNEE, SURGICAL; WITH MENISCECTOMY...",29880,37956.17,Los Robles Hospital - Thousand Oaks Surgical H...,141,Los Robles Hospital - Thousand Oaks Surgical H...,82.0,"2190 Lynn Rd Suite 100, Thousand Oaks, CA",2190 Lynn Rd Suite 100,Thousand Oaks,CA,91360,71,1.0,New Procedure,Arthroscopy
9133,9040,106564121,14176,"STRABISMUS SURGERY, RECESSION OR RESECTION PRO...",67311,27980.73,Los Robles Hospital - Thousand Oaks Surgical H...,141,Los Robles Hospital - Thousand Oaks Surgical H...,82.0,"2190 Lynn Rd Suite 100, Thousand Oaks, CA",2190 Lynn Rd Suite 100,Thousand Oaks,CA,91360,286,1.0,New Procedure,Miscellaneous
9134,9041,106564121,14177,"OPEN TREATMENT OF METATARSAL FRACTURE, WITH OR...",28485,121468.90,Los Robles Hospital - Thousand Oaks Surgical H...,141,Los Robles Hospital - Thousand Oaks Surgical H...,82.0,"2190 Lynn Rd Suite 100, Thousand Oaks, CA",2190 Lynn Rd Suite 100,Thousand Oaks,CA,91360,60,1.0,New Procedure,Miscellaneous
9135,9042,106564121,14174,"HALLUX VALGUS (BUNION) CORRECTION, WITH OR WIT...",28296,69378.73,Los Robles Hospital - Thousand Oaks Surgical H...,141,Los Robles Hospital - Thousand Oaks Surgical H...,82.0,"2190 Lynn Rd Suite 100, Thousand Oaks, CA",2190 Lynn Rd Suite 100,Thousand Oaks,CA,91360,58,1.0,New Procedure,Miscellaneous


## Find Missing Rows: 
Rows with no procedure description or CPT Code were dropped when we grouped by these columns. 

In [55]:
### Select Rows with Procedure Description NaN
missing_PD=Med_Hacks_df_updated[Med_Hacks_df_updated['Procedure Description'].isnull()]
missing_PD['Procedure Description'] = missing_PD['Procedure Description'].fillna("Missing Description")
missing_PD

/Users/james/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,Unnamed: 0,OSHPD_NUM,X1.x,Procedure Description,CPT_CODE_OR_OTHER,Top_25_Avg,Hospital_Name.x,X1.y,Hospital_Name.y,Freq,Full_Address,Address,City,State,Zip Code
2042,2043,106190049,16401,Missing Description,76700,1588.65,KINDRED BALDWIN PARK,129,KINDRED BALDWIN PARK,67.0,"14148 Francisquito Ave, Baldwin Park, CA",14148 Francisquito Ave,Baldwin Park,CA,91706


In [56]:
### Select rows with CPT NaN
missing_CPT=Med_Hacks_df_updated[Med_Hacks_df_updated['CPT_CODE_OR_OTHER'].isnull()]
missing_CPT['CPT_CODE_OR_OTHER'] = missing_CPT['CPT_CODE_OR_OTHER'].fillna("Missing CPT")
missing_CPT

/Users/james/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,Unnamed: 0,OSHPD_NUM,X1.x,Procedure Description,CPT_CODE_OR_OTHER,Top_25_Avg,Hospital_Name.x,X1.y,Hospital_Name.y,Freq,Full_Address,Address,City,State,Zip Code
60,61,190232,2553,ADJUNCTIVE THERAPY GROUP,Missing CPT,154.20,Del Amo Hospital,69,Del Amo Hospital,67.0,"23700 Camino Del Sol, Torrance, CA",23700 Camino Del Sol,Torrance,CA,90505
61,62,190232,2545,IOP MULTI/FAMILY GROUP,Missing CPT,110.00,Del Amo Hospital,69,Del Amo Hospital,67.0,"23700 Camino Del Sol, Torrance, CA",23700 Camino Del Sol,Torrance,CA,90505
62,63,190232,2544,IOP INTERACTIVE GROUP THERAPY,Missing CPT,110.00,Del Amo Hospital,69,Del Amo Hospital,67.0,"23700 Camino Del Sol, Torrance, CA",23700 Camino Del Sol,Torrance,CA,90505
63,64,190232,2549,GROUP THERAPY,Missing CPT,150.00,Del Amo Hospital,69,Del Amo Hospital,67.0,"23700 Camino Del Sol, Torrance, CA",23700 Camino Del Sol,Torrance,CA,90505
64,65,190232,2546,IOP INDIVIDUAL PSYCH,Missing CPT,110.00,Del Amo Hospital,69,Del Amo Hospital,67.0,"23700 Camino Del Sol, Torrance, CA",23700 Camino Del Sol,Torrance,CA,90505
65,66,190232,2547,PSYCH EDUCATION GROUP,Missing CPT,150.00,Del Amo Hospital,69,Del Amo Hospital,67.0,"23700 Camino Del Sol, Torrance, CA",23700 Camino Del Sol,Torrance,CA,90505
66,67,190232,2548,ADJUNCTIVE THERAPY GROUP,Missing CPT,150.00,Del Amo Hospital,69,Del Amo Hospital,67.0,"23700 Camino Del Sol, Torrance, CA",23700 Camino Del Sol,Torrance,CA,90505
67,68,190232,2550,MPD COGNITIVE THERAPY GROUP,Missing CPT,154.20,Del Amo Hospital,69,Del Amo Hospital,67.0,"23700 Camino Del Sol, Torrance, CA",23700 Camino Del Sol,Torrance,CA,90505
68,69,190232,2551,MODULE GROUP,Missing CPT,154.20,Del Amo Hospital,69,Del Amo Hospital,67.0,"23700 Camino Del Sol, Torrance, CA",23700 Camino Del Sol,Torrance,CA,90505
69,70,190232,2552,TRAUMP GROUP,Missing CPT,154.20,Del Amo Hospital,69,Del Amo Hospital,67.0,"23700 Camino Del Sol, Torrance, CA",23700 Camino Del Sol,Torrance,CA,90505


In [57]:
####Combine missing rows into a single dataframe
missing_rows=missing_PD.append(missing_CPT)
del missing_rows['Unnamed: 0']
missing_rows

,OSHPD_NUM,X1.x,Procedure Description,CPT_CODE_OR_OTHER,Top_25_Avg,Hospital_Name.x,X1.y,Hospital_Name.y,Freq,Full_Address,Address,City,State,Zip Code
2042,106190049,16401,Missing Description,76700,1588.65,KINDRED BALDWIN PARK,129,KINDRED BALDWIN PARK,67.0,"14148 Francisquito Ave, Baldwin Park, CA",14148 Francisquito Ave,Baldwin Park,CA,91706
60,190232,2553,ADJUNCTIVE THERAPY GROUP,Missing CPT,154.20,Del Amo Hospital,69,Del Amo Hospital,67.0,"23700 Camino Del Sol, Torrance, CA",23700 Camino Del Sol,Torrance,CA,90505
61,190232,2545,IOP MULTI/FAMILY GROUP,Missing CPT,110.00,Del Amo Hospital,69,Del Amo Hospital,67.0,"23700 Camino Del Sol, Torrance, CA",23700 Camino Del Sol,Torrance,CA,90505
62,190232,2544,IOP INTERACTIVE GROUP THERAPY,Missing CPT,110.00,Del Amo Hospital,69,Del Amo Hospital,67.0,"23700 Camino Del Sol, Torrance, CA",23700 Camino Del Sol,Torrance,CA,90505
63,190232,2549,GROUP THERAPY,Missing CPT,150.00,Del Amo Hospital,69,Del Amo Hospital,67.0,"23700 Camino Del Sol, Torrance, CA",23700 Camino Del Sol,Torrance,CA,90505
64,190232,2546,IOP INDIVIDUAL PSYCH,Missing CPT,110.00,Del Amo Hospital,69,Del Amo Hospital,67.0,"23700 Camino Del Sol, Torrance, CA",23700 Camino Del Sol,Torrance,CA,90505
65,190232,2547,PSYCH EDUCATION GROUP,Missing CPT,150.00,Del Amo Hospital,69,Del Amo Hospital,67.0,"23700 Camino Del Sol, Torrance, CA",23700 Camino Del Sol,Torrance,CA,90505
66,190232,2548,ADJUNCTIVE THERAPY GROUP,Missing CPT,150.00,Del Amo Hospital,69,Del Amo Hospital,67.0,"23700 Camino Del Sol, Torrance, CA",23700 Camino Del Sol,Torrance,CA,90505
67,190232,2550,MPD COGNITIVE THERAPY GROUP,Missing CPT,154.20,Del Amo Hospital,69,Del Amo Hospital,67.0,"23700 Camino Del Sol, Torrance, CA",23700 Camino Del Sol,Torrance,CA,90505
68,190232,2551,MODULE GROUP,Missing CPT,154.20,Del Amo Hospital,69,Del Amo Hospital,67.0,"23700 Camino Del Sol, Torrance, CA",23700 Camino Del Sol,Torrance,CA,90505


# Clean Data Frame and Append Missing Rows

In [58]:
### Removing unnecessary columns not shared between the dataframes we want to append allows for a cleaner concatination. 
### We also make sure column names are consistent for columns we want to keep. 
pro_cat_merged=pro_cat_merged.drop(['Unnamed: 0_x','Hospital_Name.y','Unnamed: 0_y','Freq_y','Procedure_Group_Cat'], axis = 1)
pro_cat_merged.rename(columns = {'Freq_x':'Freq'}, inplace = True) 
pro_cat_merged

,OSHPD_NUM,X1.x,Procedure Description,CPT_CODE_OR_OTHER,Top_25_Avg,Hospital_Name.x,X1.y,Freq,Full_Address,Address,City,State,Zip Code,Pro_Cat
0,90793,1393,"Emergency Room Visit, Level 2 (low to moderate...",99282,1144.79,Barton Memorial Hospital,42,56.0,"2170 South Ave, South Lake Tahoe, CA",2170 South Ave,South Lake Tahoe,CA,96150,"Emergency Room Visit, Level 2 (low to moderate..."
1,90933,1449,"Emergency Room Visit, Level 2 (low to moderate...",99282,1169.36,MARSHALL MEDICAL CENTER,150,56.0,"1100 Marshall Way, Placerville, CA",1100 Marshall Way,Placerville,CA,95667,"Emergency Room Visit, Level 2 (low to moderate..."
2,190434,3463,"Emergency Room Visit, Level 2 (low to moderate...",99282,942.00,Kaiser Foundation Hospital- West LA,122,56.0,"6041 Cadillac Ave, Los Angeles, CA",6041 Cadillac Ave,Los Angeles,CA,90034,"Emergency Room Visit, Level 2 (low to moderate..."
3,191230,5625,"Emergency Room Visit, Level 2 (low to moderate...",99282,3817.00,"MARTIN LUTHER KING JR, COMMUNITY HOSPITAL",151,55.0,"1680 E 120th St, Los Angeles, CA",1680 E 120th St,Los Angeles,CA,90059,"Emergency Room Visit, Level 2 (low to moderate..."
4,191450,5706,"Emergency Room Visit, Level 2 (low to moderate...",99282,942.00,Kaiser Foundation Hospital- Woodland Hills,123,56.0,"5601 De Soto Ave, Woodland Hills, CA",5601 De Soto Ave,Woodland Hills,CA,91367,"Emergency Room Visit, Level 2 (low to moderate..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9132,106564121,14171,"ARTHROSCOPY, KNEE, SURGICAL; WITH MENISCECTOMY...",29880,37956.17,Los Robles Hospital - Thousand Oaks Surgical H...,141,82.0,"2190 Lynn Rd Suite 100, Thousand Oaks, CA",2190 Lynn Rd Suite 100,Thousand Oaks,CA,91360,Arthroscopy
9133,106564121,14176,"STRABISMUS SURGERY, RECESSION OR RESECTION PRO...",67311,27980.73,Los Robles Hospital - Thousand Oaks Surgical H...,141,82.0,"2190 Lynn Rd Suite 100, Thousand Oaks, CA",2190 Lynn Rd Suite 100,Thousand Oaks,CA,91360,Miscellaneous
9134,106564121,14177,"OPEN TREATMENT OF METATARSAL FRACTURE, WITH OR...",28485,121468.90,Los Robles Hospital - Thousand Oaks Surgical H...,141,82.0,"2190 Lynn Rd Suite 100, Thousand Oaks, CA",2190 Lynn Rd Suite 100,Thousand Oaks,CA,91360,Miscellaneous
9135,106564121,14174,"HALLUX VALGUS (BUNION) CORRECTION, WITH OR WIT...",28296,69378.73,Los Robles Hospital - Thousand Oaks Surgical H...,141,82.0,"2190 Lynn Rd Suite 100, Thousand Oaks, CA",2190 Lynn Rd Suite 100,Thousand Oaks,CA,91360,Miscellaneous


In [59]:
####Append Missing Rows, Remove Unnecessary Columns, and Improve Column Names
AB_1045_no_dups=pro_cat_merged.append(missing_rows)
del AB_1045_no_dups['X1.y']
del AB_1045_no_dups['Hospital_Name.y']
AB_1045_no_dups['Pro_Cat'] = AB_1045_no_dups['Pro_Cat'].fillna("Miscellaneous")
AB_1045_no_dups.rename(columns = {'Pro_Cat':'Procedure Category', 'Hospital_Name.x':'Hospital_Name','X1.x':'ID'}, inplace = True) 
AB_1045_no_dups

/Users/james/opt/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:7123: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort,


,Address,CPT_CODE_OR_OTHER,City,Freq,Full_Address,Hospital_Name,OSHPD_NUM,Procedure Category,Procedure Description,State,Top_25_Avg,ID,Zip Code
0,2170 South Ave,99282,South Lake Tahoe,56.0,"2170 South Ave, South Lake Tahoe, CA",Barton Memorial Hospital,90793,"Emergency Room Visit, Level 2 (low to moderate...","Emergency Room Visit, Level 2 (low to moderate...",CA,1144.79,1393,96150
1,1100 Marshall Way,99282,Placerville,56.0,"1100 Marshall Way, Placerville, CA",MARSHALL MEDICAL CENTER,90933,"Emergency Room Visit, Level 2 (low to moderate...","Emergency Room Visit, Level 2 (low to moderate...",CA,1169.36,1449,95667
2,6041 Cadillac Ave,99282,Los Angeles,56.0,"6041 Cadillac Ave, Los Angeles, CA",Kaiser Foundation Hospital- West LA,190434,"Emergency Room Visit, Level 2 (low to moderate...","Emergency Room Visit, Level 2 (low to moderate...",CA,942.00,3463,90034
3,1680 E 120th St,99282,Los Angeles,55.0,"1680 E 120th St, Los Angeles, CA","MARTIN LUTHER KING JR, COMMUNITY HOSPITAL",191230,"Emergency Room Visit, Level 2 (low to moderate...","Emergency Room Visit, Level 2 (low to moderate...",CA,3817.00,5625,90059
4,5601 De Soto Ave,99282,Woodland Hills,56.0,"5601 De Soto Ave, Woodland Hills, CA",Kaiser Foundation Hospital- Woodland Hills,191450,"Emergency Room Visit, Level 2 (low to moderate...","Emergency Room Visit, Level 2 (low to moderate...",CA,942.00,5706,91367
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1782,1081 N China Lake Blvd,Missing CPT,Ridgecrest,25.0,"1081 N China Lake Blvd, Ridgecrest, CA",RIDGECREST REGIONAL HOSPITAL,106150782,Miscellaneous,INSPECT UPPER INTESTINE TRACT ENDO,CA,12601.54,2382,93555
1783,1081 N China Lake Blvd,Missing CPT,Ridgecrest,25.0,"1081 N China Lake Blvd, Ridgecrest, CA",RIDGECREST REGIONAL HOSPITAL,106150782,Miscellaneous,EX LG INTESTINE VIA NAT/ART OP ENDO,CA,16424.40,2383,93555
1784,1081 N China Lake Blvd,Missing CPT,Ridgecrest,25.0,"1081 N China Lake Blvd, Ridgecrest, CA",RIDGECREST REGIONAL HOSPITAL,106150782,Miscellaneous,EXCISION LT KNEE JOINT PERQ ENDO,CA,20731.01,2384,93555
1785,1081 N China Lake Blvd,Missing CPT,Ridgecrest,25.0,"1081 N China Lake Blvd, Ridgecrest, CA",RIDGECREST REGIONAL HOSPITAL,106150782,Miscellaneous,EXTRACTION POC RETAINED NAT/ART OPG,CA,15715.57,2385,93555


In [60]:
#AB_1045_no_dups.to_csv('AB_1045_no_dups_Python.csv')

In [61]:
#missing_rows.to_csv('missing_rows.csv')